Multi-resolution registration example

# Working with big data

## Strategy 1 - downsample

Registration algorithms can be expensive, but will be much faster if working on lower resolution images.

In [1]:
import os
from itertools import chain
import numpy as np

import SimpleITK as sitk

def run_elastix(fixed_image, moving_image, params):
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    elastixImageFilter.SetParameterMap(params)
    elastixImageFilter.Execute()
    return elastixImageFilter

def print_parameters(elastix_parameters):
    for k,v in elastix_parameters.iteritems():
        print(f'{k} \n\t {v}')

In [2]:
# load fixed image
fixed_image_path='../sampleImages/jrc18_down.nrrd'
fixed_image = sitk.ReadImage(fixed_image_path)

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x7f1630fbef40> >

In [3]:
# load moving image
moving_image_path = '../sampleImages/jrc10_down.nrrd'
moving_image = sitk.ReadImage(moving_image_path)

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x7f1630fbee80> >

In [4]:
# run a simple registration
affine_params = sitk.ReadParameterFile('../elastixParameters/DefaultSmoother_Affine.txt')
elastixImageFilter = run_elastix(fixed_image, moving_image, affine_params ) 
sitk.WriteImage(elastixImageFilter.GetResultImage(), '../sampleImages/affine_result_img.nrrd')

Installing all components.
InstallingComponents was successful.

ELASTIX version: 5.000
Command line options from ElastixBase:
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ./
-threads  unspecified, so all available threads are used
Command line options from TransformBase:
-t0       unspecified, so no initial transform used

Reading images...
Reading images took 0 ms.

  A default pyramid schedule is used.
  A default pyramid schedule is used.
  The default value "false" is used instead.
  The default value "GeometricalCenter" is used instead.
Transform parameters are initialized as: [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
InitializeTransform took 0.00s
  The default value "false" is used instead.
Scales for transform parameters are: [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 1, 1, 1]
Initialization of all components (before registration) took: 0 ms.
Preparation of the image pyramids took: 94 ms.

Resolut

288	-0.893650	0.000000	67.862564	0.001119	0.3
289	-0.887898	0.000000	67.862564	0.001578	0.3
290	-0.874204	0.000000	67.862564	0.001671	0.4
291	-0.889681	0.000000	67.862564	0.001258	0.3
292	-0.906905	0.000000	67.862564	0.001188	0.3
293	-0.897769	0.000000	67.862564	0.001441	0.3
294	-0.894381	0.000000	67.862564	0.001671	0.3
295	-0.897756	0.000000	67.862564	0.001106	0.3
296	-0.901947	0.000000	67.862564	0.001183	0.3
297	-0.896476	0.000000	67.862564	0.001137	0.3
298	-0.895984	0.000000	67.862564	0.000932	0.3
299	-0.900185	0.000000	67.862564	0.001538	0.3
300	-0.893367	0.000000	67.862564	0.001190	0.3
301	-0.908603	0.000000	67.862564	0.001144	0.3
302	-0.889744	0.000000	67.862564	0.001404	0.3
303	-0.897761	0.000000	67.862564	0.001099	0.3
304	-0.902854	0.000000	67.862564	0.001170	0.3
305	-0.887543	0.000000	67.862564	0.001212	0.3
306	-0.902382	0.000000	67.862564	0.000965	0.3
307	-0.898692	0.000000	67.862564	0.001379	0.3
308	-0.895416	0.000000	67.862564	0.000915	0.3
309	-0.900558	0.000000	67.862564	0

911	-0.908015	49.779821	20.134465	0.000853	0.3
912	-0.901855	49.767279	20.138034	0.000659	0.3
913	-0.900231	49.752833	20.142146	0.000892	0.3
914	-0.904307	49.741848	20.145273	0.000951	0.3
915	-0.904892	49.746675	20.143899	0.000827	0.3
916	-0.904242	49.732077	20.148056	0.000805	0.3
917	-0.907601	49.718078	20.152044	0.000723	0.3
918	-0.905225	49.706619	20.155310	0.001023	0.3
919	-0.910864	49.807711	20.126535	0.000842	0.3
920	-0.906110	49.795774	20.129928	0.000851	0.3
921	-0.908759	49.781168	20.134082	0.000695	0.3
922	-0.899930	49.804671	20.127399	0.001288	0.3
923	-0.905161	49.790068	20.131551	0.000978	0.3
924	-0.909753	49.775810	20.135606	0.000897	0.3
925	-0.915895	49.767619	20.137937	0.000513	0.3
926	-0.899025	49.764560	20.138808	0.002016	0.3
927	-0.902366	49.760961	20.139832	0.001134	0.3
928	-0.900036	50.673449	19.883428	0.000788	0.3
929	-0.910966	51.271300	19.718946	0.000645	0.3
930	-0.907999	51.256888	19.722879	0.000697	0.3
931	-0.910750	51.245293	19.726044	0.000874	0.4
932	-0.900099

413	-0.900608	62.177639	61.580202	0.001237	0.3
414	-0.912966	63.079366	60.919773	0.001309	0.3
415	-0.908977	62.988965	60.985343	0.001156	0.3
416	-0.897667	62.899578	61.050317	0.001611	0.3
417	-0.905247	62.836445	61.096291	0.001322	0.3
418	-0.907894	62.746045	61.162242	0.000718	0.3
419	-0.900570	63.177048	60.849079	0.002278	0.3
420	-0.911781	63.086733	60.914435	0.001308	0.3
421	-0.909289	62.996334	60.979993	0.000885	0.3
422	-0.898236	62.905935	61.045691	0.001973	0.3
423	-0.902165	62.815758	61.111370	0.001125	0.3
424	-0.903010	63.017117	60.964908	0.001915	0.3
425	-0.901283	62.927716	61.029849	0.000910	0.3
426	-0.896184	62.837343	61.095636	0.001941	0.3
427	-0.900857	63.816835	60.390084	0.001478	0.3
428	-0.903617	63.726435	60.454519	0.001281	0.3
429	-0.905375	63.636186	60.518982	0.001626	0.3
430	-0.904284	63.545874	60.583629	0.001004	0.4
431	-0.906351	63.455473	60.648477	0.001240	0.3
432	-0.894754	63.365092	60.713451	0.001012	0.3
433	-0.900529	63.277092	60.776846	0.002787	0.3
434	-0.907864

  Sampling the gradients took 0.041284s
Automatic parameter estimation took 0.04s
1:ItNr	2:Metric	3a:Time	3b:StepSize	4:||Gradient||	Time[ms]
0	-0.876766	0.000000	195.369723	0.004141	46.4
1	-0.872686	0.000000	195.369723	0.004540	0.5
2	-0.877409	0.000000	195.369723	0.003795	0.4
3	-0.876124	0.000000	195.369723	0.004238	0.4
4	-0.882319	0.000000	195.369723	0.004216	0.4
5	-0.864552	0.000000	195.369723	0.003210	0.4
6	-0.889416	0.000000	195.369723	0.002320	0.4
7	-0.871309	0.996093	186.522407	0.003043	0.5
8	-0.889144	0.804755	188.159152	0.001756	0.5
9	-0.880009	0.613437	189.824698	0.001511	0.6
10	-0.903320	0.513514	190.706374	0.001538	0.5
11	-0.869517	0.412074	191.609844	0.004228	0.4
12	-0.876161	0.220741	193.337465	0.002680	0.4
13	-0.877805	1.220741	184.636699	0.002104	0.4
14	-0.882465	2.220557	176.686728	0.001880	0.4
15	-0.886757	2.075477	177.797589	0.002199	0.4
16	-0.873375	3.075453	170.412750	0.002400	0.4
17	-0.902105	2.884140	171.777765	0.000995	0.4
18	-0.876945	3.670080	166.305264	0.0021

555	-0.885888	184.887026	19.927259	0.002419	0.3
556	-0.882418	185.887026	19.830940	0.001043	0.3
557	-0.882949	185.748617	19.844216	0.001449	0.3
558	-0.891548	185.627548	19.855843	0.001432	0.3
559	-0.879466	186.426705	19.779344	0.001549	0.4
560	-0.896834	186.235875	19.797558	0.002056	0.3
561	-0.895746	186.848099	19.739243	0.001758	0.3
562	-0.879520	186.788232	19.744930	0.002319	0.4
563	-0.883228	187.787727	19.650409	0.001720	0.5
564	-0.883051	188.087995	19.622189	0.001612	0.5
565	-0.887950	188.674819	19.567272	0.002483	0.4
566	-0.891514	188.803267	19.555292	0.002727	0.5
567	-0.885240	188.611939	19.573142	0.003563	0.4
568	-0.877800	188.420962	19.590991	0.000913	0.4
569	-0.882784	189.417585	19.498200	0.002160	0.4
570	-0.882609	190.393565	19.408179	0.003111	0.4
571	-0.886438	191.390214	19.317106	0.002657	0.4
572	-0.877561	192.389788	19.226619	0.001325	0.4
573	-0.875567	192.852765	19.184995	0.002265	0.4
574	-0.884130	192.764450	19.192921	0.002853	0.4
575	-0.894122	192.573112	19.210116	0.002

  Sampling gradients ...
  Sampling the gradients took 0.037879s
Automatic parameter estimation took 0.04s
1:ItNr	2:Metric	3a:Time	3b:StepSize	4:||Gradient||	Time[ms]
0	-0.849758	0.000000	123.118170	0.005797	75.7
1	-0.861113	0.000000	123.118170	0.004619	0.4
2	-0.839672	0.000000	123.118170	0.003507	0.4
3	-0.852129	0.000000	123.118170	0.007714	0.4
4	-0.852292	1.000000	117.521892	0.006848	0.4
5	-0.861473	1.994143	112.440878	0.004279	0.4
6	-0.851770	1.342232	115.721722	0.003640	0.4
7	-0.843350	2.275856	111.079979	0.004547	0.4
8	-0.852180	1.624003	114.280466	0.005943	0.4
9	-0.869955	2.617521	109.473029	0.004749	0.4
10	-0.851645	1.965601	112.580620	0.003088	0.4
11	-0.848840	2.965533	107.883331	0.004400	0.4
12	-0.852248	2.354306	110.706845	0.006104	0.4
13	-0.852755	1.702815	113.883742	0.007495	0.4
14	-0.854040	2.702815	109.079092	0.004028	0.4
15	-0.851995	3.695323	104.695194	0.003268	0.4
16	-0.834005	3.047991	107.513414	0.004193	0.4
17	-0.851002	4.047825	103.221801	0.006192	0.4
18	-0.842315	3

380	-0.853740	88.904664	23.524767	0.004779	0.5
381	-0.851904	89.845557	23.325081	0.003197	0.4
382	-0.860185	89.193635	23.463075	0.006830	0.4
383	-0.865829	90.168901	23.257238	0.005250	0.5
384	-0.862856	89.519605	23.393873	0.002368	0.4
385	-0.857872	88.867902	23.532638	0.004925	0.5
386	-0.842865	89.366634	23.426297	0.003837	0.4
387	-0.840841	90.366634	23.215944	0.007043	0.4
388	-0.863421	89.714711	23.352647	0.002532	0.4
389	-0.853951	90.712381	23.144092	0.003189	0.4
390	-0.861743	90.065854	23.278816	0.002838	0.4
391	-0.863922	90.669091	23.153064	0.006889	0.4
392	-0.855596	90.017410	23.288974	0.005695	0.4
393	-0.850021	91.017409	23.081069	0.005404	0.4
394	-0.840800	92.017409	22.876843	0.004516	0.4
395	-0.866042	93.002860	22.679094	0.006262	0.4
396	-0.857318	94.002586	22.481943	0.007128	0.4
397	-0.842437	93.350842	22.610079	0.003153	0.4
398	-0.850067	92.795929	22.720335	0.003072	0.4
399	-0.857267	92.192719	22.841412	0.005429	0.4
400	-0.849206	91.554443	22.970942	0.004576	0.4
401	-0.830262

873	-0.857929	155.858941	14.618891	0.008778	0.4
874	-0.854113	155.207018	14.672977	0.007674	0.5
875	-0.870108	156.207018	14.590176	0.003614	0.5
876	-0.861056	155.555096	14.644050	0.004322	0.5
877	-0.861770	156.554112	14.561654	0.005840	0.4
878	-0.850325	157.553220	14.480173	0.003434	0.4
879	-0.853068	158.522388	14.402001	0.004900	0.4
880	-0.854319	159.490278	14.324769	0.005248	0.4
881	-0.867311	158.838416	14.376692	0.003262	0.4
882	-0.841188	158.186501	14.428997	0.002715	0.4
883	-0.873760	158.394512	14.412267	0.002841	0.4
884	-0.844870	157.907713	14.451482	0.004805	0.4
885	-0.850447	158.894425	14.372216	0.002452	0.4
886	-0.847798	159.582357	14.317465	0.002850	0.4
887	-0.850683	159.763993	14.303078	0.005763	0.4
888	-0.851905	160.755399	14.225061	0.007297	0.4
889	-0.849169	160.103682	14.276251	0.004297	0.4
890	-0.843745	160.908899	14.213057	0.004756	0.4
891	-0.869526	161.908894	14.135352	0.003986	0.4
892	-0.849434	161.257432	14.185877	0.007186	0.4
893	-0.839383	162.257430	14.108468	0.003

In [5]:
# print the output
transformParameterMap = elastixImageFilter.GetTransformParameterMap()
print_parameters(transformParameterMap[0])

CenterOfRotationPoint 
	 ('313.12', '145.16', '90.44')
CompressResultImage 
	 ('false',)
DefaultPixelValue 
	 ('0',)
Direction 
	 ('1', '0', '0', '0', '1', '0', '0', '0', '1')
FinalBSplineInterpolationOrder 
	 ('3',)
FixedImageDimension 
	 ('3',)
FixedInternalImagePixelType 
	 ('float',)
HowToCombineTransforms 
	 ('Compose',)
Index 
	 ('0', '0', '0')
InitialTransformParametersFileName 
	 ('NoInitialTransform',)
MovingImageDimension 
	 ('3',)
MovingInternalImagePixelType 
	 ('float',)
NumberOfParameters 
	 ('12',)
Origin 
	 ('0', '0', '0')
ResampleInterpolator 
	 ('FinalBSplineInterpolator',)
Resampler 
	 ('DefaultResampler',)
ResultImageFormat 
	 ('nrrd',)
ResultImagePixelType 
	 ('float',)
Size 
	 ('413', '192', '120')
Spacing 
	 ('1.52', '1.52', '1.52')
Transform 
	 ('AffineTransform',)
TransformParameters 
	 ('1.00483', '0.0228908', '0.0264088', '-0.0232678', '1.15878', '-0.0833437', '-0.00276513', '0.0603209', '0.794836', '4.25281', '11.5245', '-27.0182')
UseDirectionCosines 
	 ('t

In [ ]:
moving_image_hi_path = '../sampleImages/JFRCtemplate2010.nrrd'
moving_image_hi = sitk.ReadImage(moving_image_hi_path)
moving_image_hi

In [ ]:
# We want to create an image at higher resolution

transformParameterMap[0]['Size'] = ['1652', '768', '479']
transformParameterMap[0]['Spacing'] = ['0.38', '0.38', '0.38']

transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameterMap(transformParameterMap)
transformixImageFilter.SetMovingImage(moving_image_hi)
transformixImageFilter.Execute()
sitk.WriteImage(transformixImageFilter.GetResultImage(), '../sampleImages/affine_result_hires_img.nrrd')